In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

# Fukui matrix spielerei

In [2]:
neutral = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , 0)  # create a neutral molecule
ion = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/h2o_crawdad.xyz" , +1)  # create a an ion with a depleted electron

In [3]:
def C_RHF(molecule):
    
    N = molecule.numberOfElectrons()
    N_P = molecule.numberOfElectronPairs()
    
    spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spin_orbital_basis.numberOfSpatialOrbitals()
    print("N: {}\nN_P: {}\nK: {}".format(N, N_P, K))

    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
    S = spin_orbital_basis.quantizeOverlapOperator().parameters()

    environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
    solver = gqcpy.RHFSCFSolver.Plain()
    objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
    rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
    
    return rhf_parameters.expansion()

In [4]:
def C_UHF(molecule):
    N = molecule.numberOfElectrons()
    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2

    spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spinor_basis.numberOfSpatialOrbitals()
    print("N_alpha: {}\nN_beta: {}\nK: {}".format(N_alpha, N_beta, K))
    
    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule) 
    S = spinor_basis.quantizeOverlapOperator().parameters()
    
    environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S)
    solver = gqcpy.UHFSCFSolver.Plain()
    uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
    return uhf_parameters.expansion()

In [5]:
print("neutral molecule: ")
C_neutral = C_UHF(neutral)
print("\n\nion:")
C_ion = C_UHF(ion)

neutral molecule: 
N_alpha: 5
N_beta: 5
K: 7


ion:
N_alpha: 5
N_beta: 4
K: 7


In [6]:
def TM(C_ion, C_neutral):
    return C_ion.inverse().transformed(C_neutral)

In [7]:
U_alpha = TM(C_ion.alpha, C_neutral.alpha)

In [8]:
U_beta = TM(C_ion.beta, C_neutral.beta)

In [9]:
U = TM(C_ion, C_neutral)

# Dyson orbital spielerei

### Random Dyson orbital

In [10]:
K = 7

onv_basis_I = gqcpy.SpinResolvedONVBasis(K, 5, 4)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
onv_basis_J = gqcpy.SpinResolvedONVBasis(K, 5, 5)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons

In [11]:
Psi_I = gqcpy.LinearExpansion_SpinResolved.Random(onv_basis_I)
#Psi_I.coefficients()
Psi_J = gqcpy.LinearExpansion_SpinResolved.Random(onv_basis_J)
#Psi_J.coefficients()

In [12]:
d_IJ = gqcpy.DysonOrbital.TransitionAmplitudes(Psi_J, Psi_I)
d_IJ.amplitudes()

array([ 0.00772793,  0.00713306, -0.04869781,  0.03673818,  0.03034079,
       -0.00469882,  0.01650801])

In [13]:
# Andere namen verzinnen voor de componenten later.
DM_I = Psi_I.calculateSpinResolved1DM()
DM_I_OD = DM_I.orbitalDensity()

DM_J = Psi_J.calculateSpinResolved1DM()
DM_J_OD = DM_J.orbitalDensity()

In [14]:
DM_I_transformed = DM_I.transformed(U)

In [20]:
fukui_matrix_neg = (DM_J - DM_I).orbitalDensity()

In [21]:
fukui_matrix_neg

array([[ 0.07022595, -0.07553891,  0.04898653, -0.01809982,  0.02576168,
         0.00243191,  0.05569602],
       [-0.07553891,  0.09918353, -0.04084811, -0.02941051,  0.03015461,
        -0.00721007, -0.00834746],
       [ 0.04898653, -0.04084811,  0.18127051,  0.06429444, -0.02050953,
        -0.00791736,  0.03424409],
       [-0.01809982, -0.02941051,  0.06429444,  0.14906124, -0.09056702,
         0.02215735, -0.00303485],
       [ 0.02576168,  0.03015461, -0.02050953, -0.09056702,  0.15998205,
        -0.01723056,  0.06242851],
       [ 0.00243191, -0.00721007, -0.00791736,  0.02215735, -0.01723056,
         0.16206409, -0.09034485],
       [ 0.05569602, -0.00834746,  0.03424409, -0.00303485,  0.06242851,
        -0.09034485,  0.17821262]])

### FCI Dyson orbital

In [16]:
def FCI(molecule, onv_basis):
    u_spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    u_spin_orbital_basis.lowdinOrthonormalize()
    
    u_hamiltonian = gqcpy.USQHamiltonian.Molecular(u_spin_orbital_basis, molecule)
    
    solver = gqcpy.EigenproblemSolver.Dense()
    environment = gqcpy.CIEnvironment.Dense(u_hamiltonian, onv_basis)

    ci_parameters = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()
    
    return ci_parameters

In [17]:
Psi_J = FCI(neutral, onv_basis_J)
Psi_I = FCI(ion, onv_basis_I)

In [18]:
d_IJ = gqcpy.DysonOrbital.TransitionAmplitudes(Psi_J, Psi_I)
d_IJ.amplitudes()

array([ 3.87269627e-16, -2.14325442e-15, -2.17202118e-14,  3.34082558e-14,
       -9.43605291e-01,  9.04945553e-15, -1.96580681e-14])